In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import gc
import os
import sys
from collections import defaultdict

# ====================================================
# 1. CẤU HÌNH (CONFIGURATION)
# ====================================================
class Config:
    # --- Paths ---
    ESM_DIR = '/kaggle/input/cafa6-protein-embeddings-esm2'
    TRAIN_TERMS = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv'
    TEST_FASTA = '/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta'
    GO_OBO = '/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo'
    
    # Dataset mới cần add để lọc Negative (Tìm dataset chứa file này trên Kaggle)
    # Nếu không tìm thấy file này, code sẽ tự động bỏ qua bước lọc Negative
    GOA_FILE = '/kaggle/input/protein-go-annotations/goa_uniprot_all.csv'
    
    # Foldseek
    HOMOLOGY_FILE = '/kaggle/input/foldseek-cafa/foldseek_submission.tsv'
    
    # --- Model & Training ---
    NUM_LABELS = 1500
    EMBED_DIM = 1280
    BATCH_SIZE = 128
    LR = 1e-3
    EPOCHS = 10
    
    # --- Post-processing ---
    THRESHOLD = 0.01
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"⚙️ Running on device: {Config.DEVICE}")

# ====================================================
# 2. MODULE: NEGATIVE PROPAGATION (MỚI)
# ====================================================
class NegativePropagator:
    def __init__(self, obo_path, goa_path):
        self.obo_path = obo_path
        self.goa_path = goa_path
        self.children = defaultdict(list)
        self.negative_pairs = set()

    def parse_obo(self):
        """Đọc file OBO để hiểu quan hệ cha-con (Parent-Child)"""
        print("   Parsing OBO ontology...")
        term = None
        with open(self.obo_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('[Term]'):
                    term = None
                elif line.startswith('id: GO:'):
                    term = line[4:]
                elif line.startswith('is_a: GO:') and term:
                    parent = line.split()[1]
                    self.children[parent].append(term) # Parent -> Children

    def get_descendants(self, root_term):
        """Tìm tất cả con cháu của một term"""
        descendants = set()
        stack = [root_term]
        while stack:
            current = stack.pop()
            if current in self.children:
                kids = self.children[current]
                for kid in kids:
                    if kid not in descendants:
                        descendants.add(kid)
                        stack.append(kid)
        return descendants

    def load_negatives(self, target_ids):
        """Load các Negative Annotations từ GOA và lan truyền xuống con cháu"""
        if not os.path.exists(self.goa_path):
            print(f"⚠️ GOA file not found at {self.goa_path}. Skipping Negative Prop.")
            return

        print("   Loading Negative Annotations (NOT)...")
        # Đọc file lớn, chỉ lấy các cột cần thiết
        # Cấu trúc thường là: ID, GO_ID, Qualifier
        # Qualifier chứa "NOT" là negative
        target_set = set(target_ids)
        
        # Đọc từng chunk để tránh tràn RAM
        chunk_iter = pd.read_csv(self.goa_path, chunksize=500000, 
                                 usecols=['protein_id', 'go_term', 'qualifier'])
        
        count = 0
        for chunk in chunk_iter:
            # Lọc các dòng có 'NOT' và thuộc tập Test
            neg_chunk = chunk[
                (chunk['qualifier'].str.contains('NOT', na=False)) & 
                (chunk['protein_id'].isin(target_set))
            ]
            
            for _, row in neg_chunk.iterrows():
                pid = row['protein_id']
                term = row['go_term']
                
                # Thêm chính nó
                self.negative_pairs.add(f"{pid}_{term}")
                
                # Thêm tất cả con cháu (Nếu không có cha thì không có con)
                descendants = self.get_descendants(term)
                for d in descendants:
                    self.negative_pairs.add(f"{pid}_{d}")
            
            count += len(neg_chunk)
            
        print(f"   Found {count} negative roots. Expanded to {len(self.negative_pairs)} blocked pairs.")

    def filter_submission(self, df_sub):
        """Lọc bỏ các cặp negative khỏi DataFrame submission"""
        print("   Applying Negative Filter...")
        initial_len = len(df_sub)
        
        # Tạo cột key để so sánh
        df_sub['key'] = df_sub['Id'] + '_' + df_sub['Term']
        
        # Giữ lại những dòng KHÔNG nằm trong tập negative
        # Dùng set để tra cứu cực nhanh
        df_clean = df_sub[~df_sub['key'].isin(self.negative_pairs)].copy()
        
        df_clean.drop(columns=['key'], inplace=True)
        removed = initial_len - len(df_clean)
        print(f"   🚫 Removed {removed} negative predictions.")
        
        return df_clean

# ====================================================
# 3. CÁC HÀM HỖ TRỢ KHÁC (NHƯ CŨ)
# ====================================================

def compute_fmax(y_true, y_pred_probs, steps=10):
    best_f1 = 0.0
    best_threshold = 0.0
    thresholds = np.linspace(0.1, 0.6, steps)
    for t in thresholds:
        y_pred_bin = (y_pred_probs >= t).astype(int)
        score = f1_score(y_true, y_pred_bin, average='samples', zero_division=0)
        if score > best_f1:
            best_f1 = score
            best_threshold = t
    return best_f1, best_threshold

def load_safe_ids(path):
    try:
        ids = np.load(path, allow_pickle=True)
        if len(ids) > 0 and isinstance(ids[0], (bytes, np.bytes_)):
            return [i.decode('utf-8') for i in ids]
        return ids.tolist()
    except: return []

def get_train_data(train_ids_list):
    df = pd.read_csv(Config.TRAIN_TERMS, sep="\t")
    top_terms = df['term'].value_counts().index[:Config.NUM_LABELS].tolist()
    term_to_idx = {t: i for i, t in enumerate(top_terms)}
    pid_to_idx = {pid: i for i, pid in enumerate(train_ids_list)}
    y = np.zeros((len(train_ids_list), Config.NUM_LABELS), dtype=np.float32)
    df = df[df['term'].isin(top_terms) & df['EntryID'].isin(train_ids_list)]
    row_idx = df['EntryID'].map(pid_to_idx)
    col_idx = df['term'].map(term_to_idx)
    valid = row_idx.notna() & col_idx.notna()
    y[row_idx[valid].astype(int), col_idx[valid].astype(int)] = 1.0
    return y, {i: t for t, i in term_to_idx.items()}

# ====================================================
# 4. MODEL ARCHITECTURE
# ====================================================
class ResidualBlock(nn.Module):
    def __init__(self, in_features, hidden_features, dropout=0.3):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(in_features, hidden_features),
            nn.BatchNorm1d(hidden_features),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_features, in_features),
            nn.BatchNorm1d(in_features)
        )
        self.relu = nn.ReLU()
    def forward(self, x): return self.relu(x + self.block(x))

class ProteinClassifier(nn.Module):
    def __init__(self, input_dim=1280, num_classes=1500):
        super().__init__()
        self.bn_input = nn.BatchNorm1d(input_dim)
        self.layer1 = nn.Linear(input_dim, 512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.res_block = ResidualBlock(512, 256)
        self.out = nn.Linear(512, num_classes)
    def forward(self, x):
        x = self.bn_input(x)
        x = self.dropout(self.relu(self.layer1(x)))
        x = self.res_block(x)
        return self.out(x)

# ====================================================
# 5. TRAINING PIPELINE (ESM-2)
# ====================================================
def run_esm_pipeline():
    print("\n" + "="*30 + "\n🚀 TRAINING ESM-2 MODEL\n" + "="*30)
    
    # A. Load Data
    try:
        all_pids = pd.read_csv(os.path.join(Config.ESM_DIR, "protein_ids.csv"))['protein_id'].tolist()
    except:
        all_pids = load_safe_ids(os.path.join(Config.ESM_DIR, "protein_ids.npy"))
    all_embeds = np.load(os.path.join(Config.ESM_DIR, "protein_embeddings.npy"), mmap_mode='r')
    pid_to_idx_map = {pid: i for i, pid in enumerate(all_pids)}
    
    # B. Prepare Train
    df_terms = pd.read_csv(Config.TRAIN_TERMS, sep="\t")
    train_targets = set(df_terms['EntryID'].unique())
    valid_pids = [p for p in train_targets if p in pid_to_idx_map]
    
    train_indices = [pid_to_idx_map[p] for p in valid_pids]
    X = np.array([all_embeds[i] for i in train_indices])
    y, term_map = get_train_data(valid_pids)
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
    
    # C. Train
    train_ds = torch.utils.data.TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train).float())
    val_ds = torch.utils.data.TensorDataset(torch.tensor(X_val).float())
    train_dl = DataLoader(train_ds, batch_size=Config.BATCH_SIZE, shuffle=True, num_workers=0)
    val_dl = DataLoader(val_ds, batch_size=Config.BATCH_SIZE, shuffle=False, num_workers=0)
    
    model = ProteinClassifier(input_dim=X.shape[1], num_classes=Config.NUM_LABELS).to(Config.DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=Config.LR)
    criterion = nn.BCEWithLogitsLoss()
    
    for epoch in range(Config.EPOCHS):
        model.train()
        total_loss = 0
        for X_b, y_b in train_dl:
            X_b, y_b = X_b.to(Config.DEVICE), y_b.to(Config.DEVICE)
            optimizer.zero_grad()
            out = model(X_b)
            loss = criterion(out, y_b)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"   Epoch {epoch+1}/{Config.EPOCHS} | Loss: {total_loss/len(train_dl):.4f}")
        
    # D. Validation
    model.eval()
    val_probs = []
    with torch.no_grad():
        for (X_b,) in val_dl:
            X_b = X_b.to(Config.DEVICE)
            val_probs.append(torch.sigmoid(model(X_b)).cpu().numpy())
    val_probs = np.vstack(val_probs)
    fmax, thresh = compute_fmax(y_val, val_probs)
    print(f"   🔥 Validation F-Max: {fmax:.4f} (Thresh: {thresh:.2f})")
    
    del X_train, y_train, X_val, y_val, train_ds, val_ds
    gc.collect()
    
    # E. Predict Test
    print("   Predicting Test Data...")
    test_pids = []
    with open(Config.TEST_FASTA, 'r') as f:
        for line in f:
            if line.startswith('>'): test_pids.append(line.strip()[1:].split()[0])
            
    X_test = np.zeros((len(test_pids), all_embeds.shape[1]), dtype=np.float32)
    for i, pid in enumerate(test_pids):
        if pid in pid_to_idx_map:
            X_test[i] = all_embeds[pid_to_idx_map[pid]]
            
    test_ds = torch.utils.data.TensorDataset(torch.tensor(X_test).float())
    test_dl = DataLoader(test_ds, batch_size=Config.BATCH_SIZE, shuffle=False, num_workers=0)
    
    preds = []
    with torch.no_grad():
        for (X_b,) in tqdm(test_dl, desc="Inference"):
            X_b = X_b.to(Config.DEVICE)
            preds.append(torch.sigmoid(model(X_b)).cpu().numpy())
    all_preds = np.vstack(preds)
    
    # Format
    dl_results = []
    idx_to_term = term_map
    for i, pid in enumerate(tqdm(test_pids, desc="Formatting")):
        scores = all_preds[i]
        indices = np.where(scores >= Config.THRESHOLD)[0]
        for idx in indices:
            dl_results.append((pid, idx_to_term[idx], scores[idx]))
            
    df_dl = pd.DataFrame(dl_results, columns=['Id', 'Term', 'Score'])
    del all_embeds, X_test, all_preds, model
    gc.collect()
    
    return df_dl, test_pids # Trả về cả list test_pids để dùng cho filter negative

# ====================================================
# 6. MAIN EXECUTION
# ====================================================
def main():
    # 1. Train & Predict Deep Learning
    df_dl, test_pids = run_esm_pipeline()
    print(f"✅ DL Predictions: {len(df_dl)} rows")
    
    # 2. Merge Foldseek
    print("\n" + "="*30 + "\n🤝 MERGING FOLDSEEK\n" + "="*30)
    try:
        print(f"   Loading: {Config.HOMOLOGY_FILE}")
        df_hom = pd.read_csv(Config.HOMOLOGY_FILE, sep='\t', header=None, names=['Id', 'Term', 'Score_Homology'])
        
        print("   Merging...")
        df_final = pd.merge(df_dl, df_hom, on=['Id', 'Term'], how='outer')
        df_final['Score'].fillna(0, inplace=True)
        df_final['Score_Homology'].fillna(0, inplace=True)
        
        # Max Pooling Strategy
        df_final['Final_Score'] = np.maximum(df_final['Score'], df_final['Score_Homology'])
        
    except Exception as e:
        print(f"⚠️ Merge Failed: {e}. Using DL Only.")
        df_final = df_dl
        df_final['Final_Score'] = df_final['Score']

    # 3. APPLY NEGATIVE PROPAGATION (MỚI)
    print("\n" + "="*30 + "\n🛑 NEGATIVE PROPAGATION\n" + "="*30)
    
    # Khởi tạo Propagator
    neg_prop = NegativePropagator(Config.GO_OBO, Config.GOA_FILE)
    
    # Parse cấu trúc cây GO
    neg_prop.parse_obo()
    
    # Load Negative từ GOA Uniprot (chỉ load cho các protein trong tập Test)
    neg_prop.load_negatives(target_ids=test_pids)
    
    # Lọc bỏ các dự đoán sai
    if len(neg_prop.negative_pairs) > 0:
        df_final = neg_prop.filter_submission(df_final)
    else:
        print("   No negative annotations found or file missing.")

    # 4. Save
    print("\n💾 SAVING SUBMISSION...")
    submission = df_final[df_final['Final_Score'] >= Config.THRESHOLD][['Id', 'Term', 'Final_Score']]
    submission['Final_Score'] = submission['Final_Score'].round(3)
    submission.columns = ['Id', 'Term', 'Score']
    
    out_path = 'submission.tsv'
    submission.to_csv(out_path, sep='\t', header=False, index=False)
    
    size_mb = os.path.getsize(out_path) / (1024 * 1024)
    print(f"✅ DONE! File size: {size_mb:.2f} MB")

if __name__ == "__main__":
    main()

⚙️ Running on device: cuda

🚀 TRAINING ESM-2 MODEL
   Epoch 1/10 | Loss: 0.0192
   Epoch 2/10 | Loss: 0.0124
   Epoch 3/10 | Loss: 0.0119
   Epoch 4/10 | Loss: 0.0115
   Epoch 5/10 | Loss: 0.0112
   Epoch 6/10 | Loss: 0.0110
   Epoch 7/10 | Loss: 0.0107
   Epoch 8/10 | Loss: 0.0105
   Epoch 9/10 | Loss: 0.0103
   Epoch 10/10 | Loss: 0.0101
   🔥 Validation F-Max: 0.3267 (Thresh: 0.21)
   Predicting Test Data...


Inference:   0%|          | 0/1753 [00:00<?, ?it/s]

Formatting:   0%|          | 0/224309 [00:00<?, ?it/s]

✅ DL Predictions: 9901479 rows

🤝 MERGING FOLDSEEK
   Loading: /kaggle/input/foldseek-cafa/foldseek_submission.tsv
   Merging...


/tmp/ipykernel_20/3707586496.py:327: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final['Score'].fillna(0, inplace=True)
/tmp/ipykernel_20/3707586496.py:328: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '


🛑 NEGATIVE PROPAGATION
   Parsing OBO ontology...
   Loading Negative Annotations (NOT)...
   Found 5294 negative roots. Expanded to 112458 blocked pairs.
   Applying Negative Filter...
   🚫 Removed 4359 negative predictions.

💾 SAVING SUBMISSION...
✅ DONE! File size: 483.83 MB


In [2]:
# Script thực hiện Hierarchy Propagation (PHIÊN BẢN TỐI ƯU HÓA)
!pip install pronto
import pandas as pd
from pronto import Ontology
from tqdm import tqdm
import numpy as np
from joblib import Parallel, delayed
import os

print("Bắt đầu thực hiện Hierarchy Propagation (TỐI ƯU HÓA)...")

# --- 1. Tải file cấu trúc GO ---
print("Đang tải file go-basic.obo vào bộ nhớ...")
with open('/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo', 'rb') as f:
    go = Ontology(f)
print(f"Đã tải thành công Ontology với {len(go)} terms.")

# --- 2. Đọc file submission đã tạo trước đó ---
submission_file = 'submission.tsv' # Đảm bảo đây là file đã được merge với homology
print(f"Đang đọc file submission gốc: {submission_file}")
original_df = pd.read_csv(submission_file, sep='\t', header=None, names=['ProteinID', 'GOTermID', 'Confidence'])
print(f"Số lượng dự đoán ban đầu: {len(original_df)}")

# --- 3. TIỀN TÍNH TOÁN (PRE-COMPUTATION) ---
# Lấy tất cả các GO term duy nhất có trong submission của bạn
unique_terms = original_df['GOTermID'].unique()
print(f"Tiền tính toán bản đồ tổ tiên cho {len(unique_terms)} GO terms duy nhất...")

# Tạo một map: term -> {tập hợp các tổ tiên}
term_ancestors_map = {}
for term_id in tqdm(unique_terms, desc="Building Ancestor Map"):
    try:
        term = go[term_id]
        # Lấy ID của tất cả các superclasses (tổ tiên) và lưu lại
        ancestors = {ancestor.id for ancestor in term.superclasses() if ancestor.id != term_id}
        term_ancestors_map[term_id] = ancestors
    except KeyError:
        term_ancestors_map[term_id] = set() # Nếu term không có trong file obo

print("Bản đồ tổ tiên đã được tạo.")

# --- 4. XỬ LÝ SONG SONG (PARALLELIZATION) ---

def process_chunk(df_chunk):
    """
    Hàm này xử lý một phần nhỏ của DataFrame.
    Nó sẽ được chạy trên một nhân CPU riêng.
    """
    propagated_predictions = {}
    # Sử dụng itertuples() nhanh hơn nhiều so với iterrows()
    for row in df_chunk.itertuples(index=False):
        protein_id = row.ProteinID
        term_id = row.GOTermID
        confidence = row.Confidence

        # Thêm dự đoán gốc
        key = (protein_id, term_id)
        if key not in propagated_predictions or confidence > propagated_predictions[key]:
            propagated_predictions[key] = confidence
        
        # Lấy các tổ tiên từ bản đồ đã tính toán trước
        if term_id in term_ancestors_map:
            for ancestor_id in term_ancestors_map[term_id]:
                ancestor_key = (protein_id, ancestor_id)
                if ancestor_key not in propagated_predictions or confidence > propagated_predictions[ancestor_key]:
                    propagated_predictions[ancestor_key] = confidence
                    
    return propagated_predictions

# Lấy số lượng nhân CPU, -1 nghĩa là dùng tất cả
n_jobs = os.cpu_count()
print(f"Chia dữ liệu thành {n_jobs} phần và xử lý song song...")

# Chia DataFrame thành các chunk nhỏ
df_chunks = np.array_split(original_df, n_jobs)

# Chạy xử lý song song
# delayed(process_chunk)(chunk) chuẩn bị lệnh để chạy hàm process_chunk với đầu vào là chunk
parallel = Parallel(n_jobs=n_jobs, backend='multiprocessing')
results_list = parallel(delayed(process_chunk)(chunk) for chunk in tqdm(df_chunks, desc="Parallel Processing"))

# --- 5. Tổng hợp kết quả ---
print("Tổng hợp kết quả từ các nhân CPU...")
final_propagated_predictions = {}
for chunk_dict in tqdm(results_list, desc="Merging results"):
    for key, value in chunk_dict.items():
        if key not in final_propagated_predictions or value > final_propagated_predictions[key]:
            final_propagated_predictions[key] = value

print("Quá trình Propagation hoàn tất.")

# --- 6. Chuyển đổi và Lưu file ---
final_list = [{'ProteinID': k[0], 'GOTermID': k[1], 'Confidence': v} for k, v in final_propagated_predictions.items()]
final_df = pd.DataFrame(final_list)

print(f"Số lượng dự đoán sau khi propagation và deduplication: {len(final_df)}")

final_submission_file = 'submission_propagated.tsv'
final_df.to_csv(final_submission_file, sep='\t', header=False, index=False)

print(f"\\nĐã tạo file submission cuối cùng: {final_submission_file}")
print("Top 5 dòng của file cuối cùng:")
print(final_df.head())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 42.8 MB/s eta 0:00:00
Bắt đầu thực hiện Hierarchy Propagation (TỐI ƯU HÓA)...
Đang tải file go-basic.obo vào bộ nhớ...
Đã tải thành công Ontology với 48106 terms.
Đang đọc file submission gốc: submission.tsv
Số lượng dự đoán ban đầu: 22215587
Tiền tính toán bản đồ tổ tiên cho 30743 GO terms duy nhất...


Building Ancestor Map: 100%|██████████| 30743/30743 [00:03<00:00, 9476.08it/s]
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Bản đồ tổ tiên đã được tạo.
Chia dữ liệu thành 4 phần và xử lý song song...


Parallel Processing: 100%|██████████| 4/4 [00:00<00:00,  5.75it/s]


Tổng hợp kết quả từ các nhân CPU...


Merging results: 100%|██████████| 4/4 [00:21<00:00,  5.38s/it]


Quá trình Propagation hoàn tất.
Số lượng dự đoán sau khi propagation và deduplication: 39973787
\nĐã tạo file submission cuối cùng: submission_propagated.tsv
Top 5 dòng của file cuối cùng:
    ProteinID    GOTermID  Confidence
0  A0A017SE81  GO:0004497       0.019
1  A0A017SE81  GO:0016491       0.438
2  A0A017SE81  GO:0003674       0.438
3  A0A017SE81  GO:0003824       0.438
4  A0A017SE81  GO:0005515       0.039
